In [1]:
from pyspark.sql import Row, SparkSession
from pyspark.sql.functions import lit
from pyspark.sql.types import StringType

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark import sql

sc = SparkContext.getOrCreate()
sqlContext = sql.SQLContext(sc)

22/05/16 13:02:33 WARN Utils: Your hostname, Udhhavs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.29.9 instead (on interface en0)
22/05/16 13:02:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/16 13:02:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/16 13:02:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
/opt/anaconda3/lib/python3.9/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
df = sc.parallelize([Row(server_name = '101 Server',cpu_utilization=85, session_count=80),
                       Row(server_name = '101 Server',cpu_utilization=80, session_count=90),
                       Row(server_name = '102 Server',cpu_utilization=85, session_count=40),
                       Row(server_name = '103 Server',cpu_utilization=70, session_count=80),
                       Row(server_name = '104 Server',cpu_utilization=60, session_count=80)
                        ]).toDF()

In [4]:
df.show()

+-----------+---------------+-------------+
|server_name|cpu_utilization|session_count|
+-----------+---------------+-------------+
| 101 Server|             85|           80|
| 101 Server|             80|           90|
| 102 Server|             85|           40|
| 103 Server|             70|           80|
| 104 Server|             60|           80|
+-----------+---------------+-------------+



In [6]:
df_na = df.withColumn('na_col',lit(None).cast(StringType()))
df_na.show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| 101 Server|             85|           80|  null|
| 101 Server|             80|           90|  null|
| 102 Server|             85|           40|  null|
| 103 Server|             70|           80|  null|
| 104 Server|             60|           80|  null|
+-----------+---------------+-------------+------+



In [7]:
df_na.fillna("A").show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| 101 Server|             85|           80|     A|
| 101 Server|             80|           90|     A|
| 102 Server|             85|           40|     A|
| 103 Server|             70|           80|     A|
| 104 Server|             60|           80|     A|
+-----------+---------------+-------------+------+



In [8]:
df2 = df_na.fillna('A').union(df_na)
df2.show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| 101 Server|             85|           80|     A|
| 101 Server|             80|           90|     A|
| 102 Server|             85|           40|     A|
| 103 Server|             70|           80|     A|
| 104 Server|             60|           80|     A|
| 101 Server|             85|           80|  null|
| 101 Server|             80|           90|  null|
| 102 Server|             85|           40|  null|
| 103 Server|             70|           80|  null|
| 104 Server|             60|           80|  null|
+-----------+---------------+-------------+------+



In [9]:
df2.na.drop().show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| 101 Server|             85|           80|     A|
| 101 Server|             80|           90|     A|
| 102 Server|             85|           40|     A|
| 103 Server|             70|           80|     A|
| 104 Server|             60|           80|     A|
+-----------+---------------+-------------+------+



In [10]:
df2.createOrReplaceTempView('na_table')

In [13]:
spark = SparkSession.builder.appName("Udhhav").getOrCreate()
spark.sql("""
select * from na_table""").show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| 101 Server|             85|           80|     A|
| 101 Server|             80|           90|     A|
| 102 Server|             85|           40|     A|
| 103 Server|             70|           80|     A|
| 104 Server|             60|           80|     A|
| 101 Server|             85|           80|  null|
| 101 Server|             80|           90|  null|
| 102 Server|             85|           40|  null|
| 103 Server|             70|           80|  null|
| 104 Server|             60|           80|  null|
+-----------+---------------+-------------+------+



In [14]:
spark.sql("""
select * from na_table where na_col is null""").show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| 101 Server|             85|           80|  null|
| 101 Server|             80|           90|  null|
| 102 Server|             85|           40|  null|
| 103 Server|             70|           80|  null|
| 104 Server|             60|           80|  null|
+-----------+---------------+-------------+------+



In [15]:
spark.sql("""
select * from na_table where na_col is not null""").show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| 101 Server|             85|           80|     A|
| 101 Server|             80|           90|     A|
| 102 Server|             85|           40|     A|
| 103 Server|             70|           80|     A|
| 104 Server|             60|           80|     A|
+-----------+---------------+-------------+------+

